In [1]:
import numpy as np

from datetime import datetime
from datetime import timedelta  
import numpy as np
import pandas as pd
from scipy.stats import laplace
import scipy.stats
import psycopg2
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime
from datetime import timedelta  
import threading
import os
import re

In [2]:
param_dic = {
    "host"      : "localhost",
    "database"  : "bachelorBesoeg2014",
    "user"      : "postgres",
    "password"  : "password",
    "port"      : "5432"
}

def execRangeQuery(params_dic,dates):
    query = """
    SELECT
    sum(count_)
    FROM _775147
    WHERE
    time_ >= '{date0}'
    AND time_ <=  '{date1}'
    ;
    """.format(date0 = dates[0], date1 = dates[1])
    try:
        connection = psycopg2.connect(**params_dic)
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

def execQuery(params_dic,query):
    try:
        connection = psycopg2.connect(**params_dic)
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")
#"""select count(*) as number_of_dates, count(DISTINCT  count_) as distinct_count from _775147;"""
query = """select time_ from _775147;"""
result = execQuery(param_dic, query)
dates = [(date[0]) for date in result]

query = """select count_ from _775147;"""
result = execQuery(param_dic, query)
"""
print(result)
print(type(result))
print(type(result[0]))
print((str(result[0][0])))
"""
counts = [(count[0]) for count in result]

def add_missing_dates(old_dates):
    start_date = old_dates[0]
    end_date = old_dates[-1]
    all_dates = pd.date_range(start = start_date, end = end_date).to_pydatetime().tolist()
    return [(date.date()) for date in all_dates]
    
def add_missing_counts(old_counts, old_dates, new_dates):
        """Adds 0 to the list of counts where there was missing dates
        Parameters:
        old_counts (list of int): List counts for each day with 
        old_dates (list of datetime.date): List of dates that is not countious
        Returns:
        List of countious starting with the first value of 
        """
        zip_iterator = zip(old_dates, old_counts)
        missing_dict =  dict(zip_iterator)
        all_counts = np.zeros(len(new_dates))
        for i, date in enumerate(new_dates):
            val = missing_dict.get(date, 0)
            all_counts[i] = val
            
        return all_counts

all_dates = add_missing_dates(dates)
all_counts =  add_missing_counts(counts, dates, all_dates)

Executed query and closed connection.
Executed query and closed connection.


In [3]:
onlydirs = [f for f in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(os.getcwd(), f)) and f[0] != '.']
onlydirs

['Data', 'plots', 'range_queries', 'results', '__pycache__']

In [4]:
os.chdir(os.getcwd()+'/'+'range_queries/flat/')
os.getcwd()

'C:\\Users\\Jonas Friis\\Documents\\Universitet\\Bachelors-Thesis\\src\\range_queries\\flat'

In [58]:
n = np.array([256,512,1024,2048])
degrees = np.array([2,3,4])
lenght_n = 32

degree_2 = np.array([44,325,402,485])

degree_3 = np.array([217,217,296,294])
degree_4 = np.array([130,200,202,288])
n_quries = 5000
def myfunc_dis(x, b):
    return 2*b*int(np.ceil(np.log(x) / np.log(b)))**2

def sample_range_query(length, dates):
    """
    Samples two dates of with distance of lenght
    """
    while True:
        start_date_idx = np.random.randint((dates[-1]-dates[0]).days+1)
        start_date = dates[start_date_idx]
        end_date = dates[start_date_idx] + timedelta(days=(int(length-1)))
        if dates[-1] > end_date:
            return (str(start_date), str(end_date))

def load_range_queries(file):
    return pd.read_csv(file, sep='\n',header=None).to_numpy().flatten()

In [50]:
files = [f for f in os.listdir('.') if re.match(r'.+(_flat_).*\.csv', f)]
files

['range_queries_flat_N=1024_B=2_r=42.csv',
 'range_queries_flat_N=1024_B=3_r=43.csv',
 'range_queries_flat_N=1024_B=4_r=44.csv',
 'range_queries_flat_N=128_B=2_r=7.csv',
 'range_queries_flat_N=128_B=3_r=8.csv',
 'range_queries_flat_N=128_B=4_r=9.csv',
 'range_queries_flat_N=2048_B=2_r=62.csv',
 'range_queries_flat_N=2048_B=3_r=63.csv',
 'range_queries_flat_N=2048_B=4_r=64.csv',
 'range_queries_flat_N=256_B=2_r=22.csv',
 'range_queries_flat_N=256_B=3_r=23.csv',
 'range_queries_flat_N=256_B=4_r=24.csv',
 'range_queries_flat_N=32_B=2_r=4.csv',
 'range_queries_flat_N=32_B=3_r=5.csv',
 'range_queries_flat_N=32_B=4_r=6.csv',
 'range_queries_flat_N=512_B=2_r=32.csv',
 'range_queries_flat_N=512_B=3_r=33.csv',
 'range_queries_flat_N=512_B=4_r=35.csv']

In [59]:
a = load_range_queries(files[0])

In [60]:
a

array(['(2015-08-21,2015-10-01)', '(2016-08-31,2016-10-11)',
       '(2015-09-10,2015-10-21)', ..., '(2014-06-13,2014-07-24)',
       '(2015-09-19,2015-10-30)', '(2015-01-20,2015-03-02)'], dtype=object)

In [61]:
split_ratio = 50
level_indicis_split = np.array_split(a, split_ratio)

In [62]:
len(level_indicis_split[49])

100

In [63]:
print(level_indicis_split[0][99])

(2014-04-23,2014-06-03)
